In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [ ]:
# Load data
station_df = pd.read_excel('station.xlsx')
abCoffee_df = pd.read_excel('abCoffee_outlets.xlsx')
locality_price_df = pd.read_excel('final_prices.xlsx')
cafe_data_df = pd.read_excel('cafe_data.xlsx')

In [ ]:
# Helper function to calculate distance in meters between two coordinates
def calculate_distance(coord1, coord2):
    coord1_rad = [radians(_) for _ in coord1]
    coord2_rad = [radians(_) for _ in coord2]
    result = haversine_distances([coord1_rad, coord2_rad])
    return result[0][1] * 6371000  # Radius of the Earth in meters

In [ ]:
# Convert latitude and longitude to numeric values, coercing errors to NaN
cafe_data_df['latitude'] = pd.to_numeric(cafe_data_df['latitude'], errors='coerce')
cafe_data_df['longitude'] = pd.to_numeric(cafe_data_df['longitude'], errors='coerce')

# Optional: Handle rows with NaN values in latitude or longitude (e.g., by removing them)
cafe_data_df = cafe_data_df.dropna(subset=['latitude', 'longitude'])

# Now proceed with your clustering, using the corrected DataFrame
coords = cafe_data_df[['latitude', 'longitude']].values
db = DBSCAN(eps=0.3/6371., min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cafe_data_df['cluster'] = db.labels_


In [ ]:
# Print information about 5 cafes with their cluster columns
print(cafe_data_df[['name', 'cluster']].head(5))


                                              name  cluster
0                                  Chai Yaari Cafe        0
1  Blue Tokai Coffee Roasters | Hiranandani Estate        1
2                                  Dream bean cafe        2
3                       Tea Post - A Place to Talk        3
4                                 Budiezz Sandwich        4


In [ ]:
# Find out how many clusters exist
num_clusters = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)
print("Number of clusters:", num_clusters)


Number of clusters: 1597


In [ ]:
# Calculate outlet metric for each cluster
def calculate_outlet_metric(cluster):
    categories = cluster['category'].value_counts().to_dict()
    metric = sum([categories.get(cat, 0) * val for cat, val in {'Cafe': 50, 'Coffee Shop': 65, 'Tea House': 20, 'Other': 35}.items()])
    return metric

In [ ]:
# Calculate outlet metric for each cluster
metric_values = []
for cluster_id in range(num_clusters):
    cluster = cafe_data_df[cafe_data_df['cluster'] == cluster_id]
    metric_value = calculate_outlet_metric(cluster)
    metric_values.append(metric_value)

# Display the first 5 metric values
print("Metric values for the first 5 clusters:")
for i in range(5):
    print("Cluster", i, ":", metric_values[i])


Metric values for the first 5 clusters:
Cluster 0 : 430
Cluster 1 : 2495
Cluster 2 : 430
Cluster 3 : 0
Cluster 4 : 290


In [ ]:
cluster_metrics = cafe_data_df.groupby('cluster').apply(calculate_outlet_metric)


In [ ]:
# Display the DataFrame resulting from the groupby operation
print(cluster_metrics)


cluster
0        430
1       2495
2        430
3          0
4        290
        ... 
1592       0
1593       0
1594       0
1595      35
1596       0
Length: 1597, dtype: int64


In [ ]:
# Calculate distance metric for each cluster
def calculate_distance_metric(cluster_center):
    distances = station_df.apply(lambda row: calculate_distance(cluster_center, (row['latitude'], row['longitude'])), axis=1)
    nearest_station_distance = distances.min()
    return nearest_station_distance


In [ ]:
# Calculate distance metric for each cluster
distance_metrics = cafe_data_df.groupby('cluster').apply(lambda cluster: calculate_distance_metric((cluster['latitude'].mean(), cluster['longitude'].mean())))

# Display the distance metric for the top 10 clusters
print("Distance metric for the top 10 clusters:")
for i, distance_metric in enumerate(distance_metrics.head(10)):
    print(f"Cluster {i}: Distance to nearest station = {distance_metric} meters")


Distance metric for the top 10 clusters:
Cluster 0: Distance to nearest station = 5111.39571478834 meters
Cluster 1: Distance to nearest station = 5770.795977947639 meters
Cluster 2: Distance to nearest station = 4386.156998460547 meters
Cluster 3: Distance to nearest station = 5625.813030313808 meters
Cluster 4: Distance to nearest station = 4337.578390622429 meters
Cluster 5: Distance to nearest station = 3648.187842631361 meters
Cluster 6: Distance to nearest station = 1053.4948322732048 meters
Cluster 7: Distance to nearest station = 4434.210306002356 meters
Cluster 8: Distance to nearest station = 3355.684913205975 meters
Cluster 9: Distance to nearest station = 3086.6876831844193 meters


In [ ]:
# Calculate distance metric for each cluster
distance_metrics = cafe_data_df.groupby('cluster').apply(lambda cluster: calculate_distance_metric((cluster['latitude'].mean(), cluster['longitude'].mean())))

# Sort clusters based on distance metric values
distance_metrics_sorted = distance_metrics.sort_values(ascending=True)

# Display the distance metric for the top 10 clusters
print("Distance metric for the top 10 clusters:")
for i in range(10):
    print(f"Cluster {distance_metrics_sorted.index[i]}: Distance to nearest station = {distance_metrics_sorted.iloc[i]} meters")


Distance metric for the top 10 clusters:
Cluster 584: Distance to nearest station = 20.954530021404445 meters
Cluster 603: Distance to nearest station = 29.474802595555886 meters
Cluster 84: Distance to nearest station = 54.25057455092763 meters
Cluster 44: Distance to nearest station = 59.855683635704764 meters
Cluster 926: Distance to nearest station = 59.96639697432185 meters
Cluster 62: Distance to nearest station = 65.09946754570545 meters
Cluster 22: Distance to nearest station = 68.2712095874716 meters
Cluster 768: Distance to nearest station = 80.49296829723824 meters
Cluster 961: Distance to nearest station = 85.91995703403047 meters
Cluster 987: Distance to nearest station = 90.33355298996327 meters


In [ ]:
#distance of cluster from train stations
clusters_centers = cafe_data_df.groupby('cluster')[['latitude', 'longitude']].mean()
distance_metrics = clusters_centers.apply(lambda row: calculate_distance_metric((row['latitude'], row['longitude'])), axis=1)


In [ ]:
# Calculate locality price metric for each cluster
def calculate_price_metric(cluster_center):
    distances = locality_price_df.apply(lambda row: calculate_distance(cluster_center, (row['latitude'], row['longitude'])), axis=1)
    nearest_locality_index = distances.idxmin()
    return locality_price_df.loc[nearest_locality_index, 'price_per_sqft']


In [ ]:

price_metrics = clusters_centers.apply(lambda row: calculate_price_metric((row['latitude'], row['longitude'])), axis=1)
# Sort the price metrics Series
price_metrics_sorted = price_metrics.sort_values(ascending=True)

# Display the top 5 price metrics
print("Top 5 price metrics for clusters:")
print(price_metrics_sorted.head(5))


Top 5 price metrics for clusters:
cluster
417     1,07,515.91 / sqft
1021          1,500 / sqft
1020          1,500 / sqft
153           1,500 / sqft
961           1,500 / sqft
dtype: object


### ABCOFFEE


In [ ]:
# Convert latitude and longitude in abCoffee_df to numeric values
abCoffee_df['latitude'] = pd.to_numeric(abCoffee_df['latitude'], errors='coerce')
abCoffee_df['longitude'] = pd.to_numeric(abCoffee_df['longitute'], errors='coerce')


In [ ]:
# Function to create clusters around abCoffee outlets and calculate metrics
def calculate_abCoffee_metrics(row):
    # Identify coffee outlets within 100m radius
    center = (row['latitude'], row['longitude'])
    nearby_outlets = cafe_data_df.apply(lambda x: calculate_distance(center, (x['latitude'], x['longitude'])) < 300, axis=1)
    cluster = cafe_data_df[nearby_outlets]

     # Calculate outlet metric for the cluster
    outlet_metric = calculate_outlet_metric(cluster)

    # Calculate distance metric (distance to nearest railway station)
    distance_metric = calculate_distance_metric(center)

    # Calculate price metric (nearest locality price per square foot)
    price_metric = calculate_price_metric(center)

    return outlet_metric, distance_metric, price_metric


In [ ]:
# Apply the function to each abCoffee outlet
abCoffee_metrics = abCoffee_df.apply(calculate_abCoffee_metrics, axis=1, result_type='expand')
abCoffee_metrics.columns = ['Outlet Metric', 'Distance Metric', 'Price Metric']



In [ ]:
print(abCoffee_metrics)


   Outlet Metric  Distance Metric      Price Metric
0             50      3468.344931   9,844.56 / sqft
1             50      1991.067060  29,119.67 / sqft
2            100      2285.632522  97,331.79 / sqft
3            680      3371.183107  18,333.33 / sqft
4            380      3462.051023   8,253.11 / sqft
5              0      3033.444586  19,777.78 / sqft
6              0      2180.138909  73,875.62 / sqft
7              0      1924.999316     15,865 / sqft
8             70       565.820135   33,153.3 / sqft
9              0      1059.167366   9,285.14 / sqft


In [ ]:
# Convert the 'Price Metric' column to numeric values
abCoffee_metrics['Price Metric'] = abCoffee_metrics['Price Metric'].str.replace(',', '')  # Remove commas
abCoffee_metrics['Price Metric'] = abCoffee_metrics['Price Metric'].str.extract(r'([\d.]+)').astype(float)  # Extract numeric values

# Calculate the mean for all metrics
ideal_metrics = abCoffee_metrics.mean()

print(ideal_metrics)

Outlet Metric        133.000000
Distance Metric     2334.184896
Price Metric       31483.930000
dtype: float64


In [ ]:
# to be run once
# Convert the 'Price Metric' column to numeric values
price_metrics = price_metrics.str.replace(',', '')  # Remove commas
price_metrics = price_metrics.str.extract(r'([\d.]+)').astype(float)  # Extract numeric values

In [ ]:
print(price_metrics)
print(ideal_metrics)


                0
cluster          
0        16504.30
1        16504.30
2        10000.00
3        16504.30
4        10000.00
...           ...
1592      6912.90
1593      4929.58
1594      5953.80
1595      6912.90
1596      4929.58

[1597 rows x 1 columns]
Outlet Metric        133.000000
Distance Metric     2334.184896
Price Metric       31483.930000
dtype: float64


In [ ]:
# Function to calculate the weighted difference between two sets of metrics
def calculate_weighted_difference(cluster_metrics, ideal_metrics):
    # Ensure all metrics are in float format
    outlet_diff = np.abs(float(cluster_metrics.get('Outlet Metric', 0)) - float(ideal_metrics.get('Outlet Metric', 0))) * 0.3
    distance_diff = np.abs(float(cluster_metrics.get('Distance Metric', 0)) - float(ideal_metrics.get('Distance Metric', 0))) * 0.3
    price_diff = np.abs(float(cluster_metrics.get('Price Metric', 0)) - float(ideal_metrics.get('Price Metric', 0))) * 0.4
    weighted_diff = outlet_diff + distance_diff + price_diff
    return weighted_diff

In [ ]:
# Calculate the weighted differences for each cluster with the updated function
weighted_differences = clusters_centers.apply(lambda x: calculate_weighted_difference(
    {'Outlet Metric': float(cluster_metrics.loc[x.name]), 'Distance Metric': float(distance_metrics.loc[x.name]), 'Price Metric': price_metrics.loc[x.name]},
    ideal_metrics), axis=1)


In [ ]:
# Proceed with finding the best cluster for a new outlet as before
best_cluster_idx = weighted_differences.idxmin()
best_location = clusters_centers.loc[best_cluster_idx]

In [ ]:
# Sort the clusters by their weighted differences in ascending order
sorted_clusters = weighted_differences.sort_values()

# Select the top 10 clusters
top_10_clusters = sorted_clusters.head(10)

# Retrieve the locations of the top 10 clusters
top_10_locations = clusters_centers.loc[top_10_clusters.index]


In [ ]:
# Print the recommended locations for new outlets
print("Recommended locations for new outlets:")
for idx, location in top_10_locations.iterrows():
    print(f"Cluster {idx}: Latitude {location['latitude']}, Longitude {location['longitude']}")

Recommended locations for new outlets:
Cluster 1382: Latitude 19.0983353, Longitude 72.9057331
Cluster 117: Latitude 19.095561433333334, Longitude 72.90257153333333
Cluster 824: Latitude 19.0848189, Longitude 72.8594694
Cluster 1317: Latitude 19.0894375, Longitude 72.8986824
Cluster 338: Latitude 19.1011684, Longitude 72.8562889
Cluster 1261: Latitude 19.1391857, Longitude 72.8667238
Cluster 1390: Latitude 19.0905609, Longitude 72.9025301
Cluster 846: Latitude 19.132869025, Longitude 72.858133125
Cluster 1405: Latitude 19.1367532, Longitude 72.8579386
Cluster 86: Latitude 19.087742, Longitude 72.88955179615385


In [ ]:
import folium
import pandas as pd

# Load the recommended locations
# Assuming top_10_locations is already defined


# Create a map object centered around an average location of the recommended locations
map_center_latitude = top_10_locations['latitude'].mean()
map_center_longitude = top_10_locations['longitude'].mean()
m = folium.Map(location=[map_center_latitude, map_center_longitude], zoom_start=12)

# Add points for each recommended location
for idx, location in top_10_locations.iterrows():
    folium.Marker(
        [location['latitude'], location['longitude']],
        popup=f"Recommended Cluster {idx}",
        icon=folium.Icon(color='green')
    ).add_to(m)

m

In [ ]:
# Print the metrics of the top 10 recommended clusters
print("Metrics of the top 10 recommended clusters:")
for cluster_id, location in top_10_locations.iterrows():
    outlet_metric = cluster_metrics.loc[cluster_id]
    distance_metric = distance_metrics.loc[cluster_id]
    price_metric = price_metrics.loc[cluster_id]
    weighted_diff = weighted_differences.loc[cluster_id]

    print(f"Cluster {cluster_id}:")
    print(f"Outlet Metric: {outlet_metric}")
    print(f"Distance Metric: {distance_metric} meters")
    print(f"Price Metric: {price_metric}")
    print(f"Weighted Difference: {weighted_diff}")
    print("--------------------------------------")


Metrics of the top 10 recommended clusters:
Cluster 1382:
Outlet Metric: 35
Distance Metric: 1432.7455510814177 meters
Price Metric: 0    31185.01
Name: 1382, dtype: float64
Weighted Difference: 419.39980336136745
--------------------------------------
Cluster 117:
Outlet Metric: 70
Distance Metric: 1254.9792066007162 meters
Price Metric: 0    31185.01
Name: 117, dtype: float64
Weighted Difference: 462.2297067055779
--------------------------------------
Cluster 824:
Outlet Metric: 35
Distance Metric: 2331.1498341452793 meters
Price Metric: 0    32723.21
Name: 824, dtype: float64
Weighted Difference: 526.0225184422077
--------------------------------------
Cluster 1317:
Outlet Metric: 0
Distance Metric: 1097.8055818767496 meters
Price Metric: 0    31185.01
Name: 1317, dtype: float64
Weighted Difference: 530.3817941227678
--------------------------------------
Cluster 338:
Outlet Metric: 50
Distance Metric: 1298.3756602744113 meters
Price Metric: 0    32000.0
Name: 338, dtype: float64
W